In [8]:
import random
import numpy as np
from ase import Atoms
from pymatgen.transformations.standard_transformations import SupercellTransformation
from pymatgen.io.ase import AseAtomsAdaptor
from ase.geometry.analysis import Analysis
from chgnet.model.model import CHGNet
from chgnet.model.dynamics import MolecularDynamics
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count
import multiprocessing
import sys
from dask import delayed
from dask.distributed import Client, LocalCluster
import dask

sys.path.append("../")
from core import RotateStructure
from utils import create_spehircal_mesh_points

In [26]:
def get_structure(theta):
    rs = RotateStructure()
    return rs.get_rotated_structure(phi=0, theta=theta, remove_Rb=False, add_charge=False)

print(get_structure(0),get_structure(180))

Full Formula (Rb2 Ge2 Cl6)
Reduced Formula: RbGeCl3
abc   :   5.914370   7.196420   8.361430
angles:  90.000000 107.593180  90.000000
Sites (10)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Rb    0.80552   0         0.373834
  1  Rb    0.15114   0.5       0.972426
  2  Ge    0.550001  0         0.799997
  3  Ge    0.406659  0.5       0.546263
  4  Cl    0.147697  0         0.790532
  5  Cl    0.808963  0.5       0.555728
  6  Cl    0.295171  0.257524  0.349633
  7  Cl    0.661489  0.757524  0.996627
  8  Cl    0.295171  0.742476  0.349633
  9  Cl    0.661489  0.242476  0.996627 Full Formula (Rb2 Ge2 Cl6)
Reduced Formula: RbGeCl3
abc   :   5.914370   7.196420   8.361430
angles:  90.000000 107.593180  90.000000
Sites (10)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Rb    0.80552   0         0.373834
  1  Rb    0.15114   0.5       0.972426
  2  Ge    0.550001  0         0.799997
  3  Ge    0.496323  0.5      

In [9]:
from pymatgen.transformations.standard_transformations import SupercellTransformation

def create_supercell(structure, scaling_matrix=None):
    """Create a supercell from a given structure."""
    if scaling_matrix is None:
        scaling_matrix = [[2, 0, 0], [0, 2, 0], [0, 0, 2]]
    
    transformation = SupercellTransformation(scaling_matrix)
    supercell = transformation.apply_transformation(structure)
    
    return supercell


In [10]:
def create_mixed_supercell(structure0, structure180, swap_fraction=0.5):
    """Create a mixed supercell by swapping a fraction of atom positions."""
    num_atoms = len(structure0)
    num_swaps = int(swap_fraction * num_atoms)
    
    # Randomly choose atom indices for swapping
    swap_indices = random.sample(range(num_atoms), num_swaps)
    
    # Create a deep copy of structure0 to avoid modifying the original structure
    mixed_supercell = structure0.copy()
    
    # Perform the swaps
    for idx in swap_indices:
        mixed_supercell[idx].coords = structure180[idx].coords
    
    return mixed_supercell


In [11]:
def run_md_at_temperature(structure, chgnet_model, temperature, timestep, trajectory_base, logfile_base, loginterval, use_device):
    md = MolecularDynamics(
        atoms=structure,
        model=chgnet_model,
        ensemble="nvt",
        temperature=temperature,
        timestep=timestep,
        trajectory=f"../data/md_files/{trajectory_base}_T_{temperature}.traj",
        logfile=f"../data/md_files/{logfile_base}_T_{temperature}.log",
        loginterval=loginterval,
        use_device=use_device,
    )
    md.run(50)  # Run each simulation for a fixed duration
    return md.atoms

In [12]:
from ase.geometry.analysis import Analysis

def compute_rdf(structure, rmax=10, n_bins=100):
    """Compute the RDF for a given structure."""
    
    # Check if structure is an ASE Atoms object or a Pymatgen Structure
    if isinstance(structure, Atoms):
        atoms = structure
    else:
        # Convert Pymatgen structure to ASE Atoms
        lattice = structure.lattice.matrix
        atoms = Atoms(
            symbols=[site.species_string for site in structure],
            positions=[site.coords for site in structure],
            cell=lattice
        )
    
    analysis = Analysis(atoms)
    rdf_data = analysis.get_rdf(rmax=rmax, nbins=n_bins, return_dists=True)
    
    r = rdf_data[0][1]  # Extracting distances
    rdf = rdf_data[0][0]  # Extracting rdf values
    
    return r, rdf


In [15]:
def rdf_difference(rdf1, rdf2):
    """Calculate the difference between two RDFs."""
    return np.linalg.norm(rdf1 - rdf2)

def get_structure(theta):
    rs = RotateStructure()
    return rs.get_rotated_structure(phi=0, theta=theta, remove_Rb=False, add_charge=False)

def create_mixed_supercell(structure0, structure180, swap_fraction=0.5):
    """Create a mixed supercell by swapping a fraction of atom positions."""
    num_atoms = structure0.num_sites  # Modified here
    num_swaps = int(swap_fraction * num_atoms)
    
    # Randomly choose atom indices for swapping
    swap_indices = random.sample(range(num_atoms), num_swaps)
    
    # Create a deep copy of structure0 to avoid modifying the original structure
    mixed_structure = structure0.copy()
    
    # Perform the swaps
    for idx in swap_indices:
        mixed_structure.replace(idx, structure180[idx].species_string, coords=structure180[idx].coords)
    
    return mixed_structure


structure0 = get_structure(0)
structure180 = get_structure(180)

ss=create_mixed_supercell(structure0, structure180, swap_fraction=0.5)

In [17]:
from

['_lattice',
 '_sites',
 '_charge',
 '__module__',
 '__doc__',
 '__hash__',
 '__init__',
 '__setitem__',
 '__delitem__',
 'lattice',
 'append',
 'insert',
 'replace',
 'substitute',
 'remove_species',
 'remove_sites',
 'apply_operation',
 'apply_strain',
 'sort',
 'translate_sites',
 'rotate_sites',
 'perturb',
 'make_supercell',
 'scale_lattice',
 'merge_sites',
 'set_charge',
 '__abstractmethods__',
 '_abc_impl',
 'from_sites',
 'from_spacegroup',
 'from_magnetic_spacegroup',
 'charge',
 'distance_matrix',
 'sites',
 'density',
 'get_space_group_info',
 'matches',
 '__eq__',
 '__ne__',
 '__mul__',
 '__rmul__',
 'frac_coords',
 'volume',
 'get_distance',
 'get_sites_in_sphere',
 'get_neighbors',
 'get_neighbors_old',
 '_get_neighbor_list_py',
 'get_neighbor_list',
 'get_symmetric_neighbor_list',
 'get_all_neighbors',
 'get_all_neighbors_py',
 'get_all_neighbors_old',
 'get_neighbors_in_shell',
 'get_sorted_structure',
 'get_reduced_structure',
 'copy',
 'interpolate',
 'get_miller_ind

In [24]:
from chgnet.model.model import CHGNet
from chgnet.model.dynamics import MolecularDynamics
import matplotlib.pyplot as plt

chgnet = CHGNet.load()

# Get the structures for 0 and 180 configurations
structure0 = get_structure(0)
structure180 = get_structure(180)

# Create supercells for both configurations
supercell0 = create_supercell(structure0)
supercell180 = create_supercell(structure180)

# Create the mixed supercell by swapping a fraction of atom positions
mixed_supercell = create_mixed_supercell(supercell0, supercell180, swap_fraction=0.5)

# Convert the mixed supercell to ASE Atoms object
mixed_atoms = pymatgen_structure_to_ase_atoms(mixed_supercell)

# Running the MD and analysis
rdf_temps = [50, 500, 1000, 1500, 2000, 2500]
rdf_results = []

for temp in rdf_temps:
    final_structure = run_md_at_temperature(mixed_atoms, chgnet, temp, 2, "md_mixed", "log_mixed", 100, "cpu")
    _, rdf_temp = compute_rdf(final_structure, rmax=2.5, n_bins=100)
    rdf_results.append(rdf_temp)

# Plot the RDFs
plt.figure(figsize=(10,6))
r, _ = compute_rdf(supercell0, rmax=10, n_bins=100)  # Compute RDF for supercell0 for plotting purpose
plt.plot(r, rdf0, label="RDF0", color='blue')
plt.plot(r, rdf180, label="RDF180", color='red')

for i, rdf_temp in enumerate(rdf_results):
    plt.plot(r, rdf_temp, label=f"Temp = {rdf_temps[i]}K")

plt.xlabel("Distance (r)")
plt.ylabel("g(r)")
plt.title("Radial Distribution Functions")
plt.legend()
plt.show()


CHGNet initialized with 400,438 parameters
CHGNet will run on cpu


/opt/homebrew/Caskroom/miniconda/base/envs/dft/lib/python3.8/site-packages/ase/md/nvtberendsen.py:95: RuntimeWarning: divide by zero encountered in double_scalars
  (self.temperature / old_temperature - 1.0) *


CHGNet will run on cpu
CHGNet will run on cpu
CHGNet will run on cpu
CHGNet will run on cpu
CHGNet will run on cpu


NameError: name 'rdf0' is not defined

<Figure size 720x432 with 0 Axes>

In [28]:
def rdf_difference(rdf1, rdf2):
    """Calculate the difference between two RDFs."""
    return np.linalg.norm(rdf1 - rdf2)
def create_mixed_supercell(structure0, structure180, swap_fraction=0.5):
    """Create a mixed supercell by swapping a fraction of atom positions."""
    num_atoms = structure0.num_sites  # Modified here
    num_swaps = int(swap_fraction * num_atoms)
    
    # Randomly choose atom indices for swapping
    swap_indices = random.sample(range(num_atoms), num_swaps)
    
    # Create a deep copy of structure0 to avoid modifying the original structure
    mixed_structure = structure0.copy()
    
    # Perform the swaps
    for idx in swap_indices:
        mixed_structure.replace(idx, structure180[idx].species_string, coords=structure180[idx].coords)
    
    return mixed_structure


structure0 = get_structure(0)
structure180 = get_structure(180)

# Create supercells for both configurations
supercell0 = structure0.make_supercell([2, 2, 2])
supercell180 = structure180.make_supercell([2, 2, 2])
supercell0 = structure0.copy()
supercell0.make_supercell([2, 2, 2])

supercell180 = structure180.copy()
supercell180.make_supercell([2, 2, 2])

# Create the mixed supercell
mixed_supercell = create_mixed_supercell(supercell0, supercell180, swap_fraction=0.5)

# Convert the mixed supercell to ASE Atoms object
mixed_atoms = pymatgen_structure_to_ase_atoms(mixed_supercell)

# Calculate RDFs for the reference structures
r, rdf_0 = compute_rdf(structure0, rmax=10, n_bins=100)
_, rdf_180 = compute_rdf(structure180, rmax=10, n_bins=100)

# Running the MD and analysis
rdf_temps = [50, 500, 1000, 1500, 2000, 2500]
rdf_differences = []

for temp in rdf_temps:
    final_structure = run_md_at_temperature(mixed_atoms, chgnet, temp, 2, "md_mixed", "log_mixed", 100, "cpu")
    _, rdf_temp = compute_rdf(final_structure, rmax=2.5, n_bins=100)
    
    diff_0 = rdf_difference(rdf_temp, rdf_0)
    diff_180 = rdf_difference(rdf_temp, rdf_180)
    
    # Store the minimum difference (either to 0 or to 180)
    rdf_differences.append(min(diff_0, diff_180))


CHGNet will run on cpu


In [ ]:
# Plot the RDF differences against temperature
plt.plot(rdf_temps, rdf_differences, marker='o')
plt.xlabel('Temperature (K)')
plt.ylabel('RDF Difference')
plt.title('RDF Difference vs. Temperature')
plt.show()

In [5]:
structure0 = get_structure(0)
structure180 = get_structure(180)

# Create supercells for both configurations
supercell0 = create_supercell(structure0)
supercell180 = create_supercell(structure180)

# Create the mixed supercell by swapping a fraction of atom positions
mixed_supercell = create_mixed_supercell(supercell0, supercell180, swap_fraction=0.5)

NameError: name 'get_structure' is not defined

In [2]:
from pymatgen.io.cif import CifParser

In [18]:
structure0 = get_structure(0)
structure180 = get_structure(180)

# Create supercells for both configurations
supercell0 = create_supercell(structure0)
supercell180 = create_supercell(structure180)

# Create the mixed supercell by swapping a fraction of atom positions
mixed_supercell = create_mixed_supercell(supercell0, supercell180, swap_fraction=0.5)

In [19]:
from pymatgen.io import cif

In [24]:
cif.CifWriter(mixed_supercell).write_file("mixed_supercell.cif")

In [28]:
import numpy as np
from pymatgen.core import Structure, Lattice, DummySpecie, PeriodicSite

# Load the two structures you provided
structure0 = get_structure(0)
structure180 = get_structure(180)

# Create a mixed supercell with a boundary
boundary_thickness = 1.0  # Thickness of the boundary in Angstroms
lattice_params = np.array([5.914370, 7.196420, 8.361430])  # ABC lattice parameters
mixed_lattice = Lattice.from_parameters(*lattice_params)

# Determine the number of unit cells in the supercell along each direction
num_cells = np.ceil(lattice_params / (2 * boundary_thickness)).astype(int)

# Create the supercell with alternating slices of structure0 and structure180
mixed_structure = Structure.from_sites([])  # Initialize an empty structure
for i in range(num_cells[0]):
    for j in range(num_cells[1]):
        for k in range(num_cells[2]):
            if (i + j + k) % 2 == 0:
                shift = np.array([i, j, k]) * (2 * boundary_thickness)
                for site in structure0:
                    site_coords = site.coords + shift
                    mixed_structure.append(PeriodicSite(site.species_and_occu, site_coords, mixed_lattice))

            else:
                shift = np.array([i, j, k]) * (2 * boundary_thickness)
                for site in structure180:
                    site_coords = site.coords + shift
                    mixed_structure.append(PeriodicSite(site.species_and_occu, site_coords, mixed_lattice))

# Now 'mixed_structure' contains the mixed supercell with a wall boundary


ImportError: cannot import name 'DummySpecie' from 'pymatgen.core' (/opt/homebrew/Caskroom/miniconda/base/envs/dft/lib/python3.8/site-packages/pymatgen/core/__init__.py)